In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Data Visualisation of criminal data of India from 2001-2012

We are visualising the criminal data of India initially we plot some basic plots then we make a PLOTLY DASH then make some animated plots after reading this notebook you will be able to use plotly dash p, animated plots . And you will get a very good knowledge of Crimes in India

Import some useful libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline



import plotly.express as px
import plotly.graph_objects as go

Import dataframe
data source : https://data.world/rajanand/crime-in-india/workspace/file?filename=01_District_wise_crimes_committed_IPC_2001_2012.csv

In [ ]:
df = pd.read_csv("../input/district-wise-crimes-in-india/01_District_wise_crimes_committed_IPC_2001_2012.csv")
df.head()


First we will see how is the total number of IPC crimes is increasing year by year in India for this we will plot a line plot

In [ ]:
df_by_year = df.groupby(by='YEAR').sum()
df_by_year.head()

In [ ]:
df_by_year.plot(kind='line',y='TOTAL IPC CRIMES')
plt.title("Bar chart of Total no of IPC crimes vs year",fontdict={'size':15,'fontfamily':'Times New Roman'})
plt.ylabel("Total No of IPC Crimes",fontdict={'size':13,'fontfamily':'DejaVu Sans'})
plt.xlabel("Year",fontdict={'size':14,'fontfamily':'DejaVu Sans'})
plt.show()

from above graph we can see that the total number of IPC crimes are increasing in continuous manner but in 2003 crime rate decrease and then in 2005 again decrease at small amount

In [ ]:
df_by_state = df.groupby(by ='STATE/UT').sum()
df_by_state.head()

In [ ]:
fig,ax=plt.subplots(figsize=(20,10))
plt.tick_params(labelsize=14)
plt.xticks(label = df_by_state.index,rotation=90)
ax.bar(x=df_by_state.index,height=df_by_state['TOTAL IPC CRIMES'])
ax.set_facecolor('0')
ax.set_title("Bar graph of total no of IPC crimes Vs States/UT",fontdict={'size':26,'fontfamily':"Britannic Bold"})
ax.set_ylabel("Total no of IPC crimes",fontdict={'size':20,'fontfamily':"DejaVu Sans"})
ax.set_xlabel("States/Union Territories",fontdict={'size':20,'fontfamily':"DejaVu Sans"})

From abve graph we can say the total crime rate in Madhya pradesh is maximum in 2001-2012

From the above two graph we can't see that which crime does most in which year and in which state  so for taking all the crimes specifically we need to plot many graphs which is not a good idea for anyone so the solution is we can make dashboard in which by choosing specific crime name we can see which crime is most done in which year for making dash board we need to import some libraries

# Plotly dash 

Import useful libraries

In [ ]:
pip install dash

In [ ]:
pip install dash_auth

In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input,Output
import dash_auth

In [ ]:
app = dash.Dash()

Crime_names = []
for i in df_by_year.columns:
    Crime_names.append({'label':i,'value':i})

username_password = [['username','password'],['Kaggle','Anas']]
auth = dash_auth.BasicAuth(app,username_password)
app.layout = html.Div([
                html.H1(
                    'Crimes in India fron 2001 to 2012'
                  
                ),
                html.Label("Enter the name of crime"),
                html.Div([
                    html.Div([
                            dcc.Dropdown(id = 'Name_of_crime',
                                options = Crime_names,
                                value = 'MURDER'
                        )],
                        style={"width":'48%','display':'inline-block'}
                    ),
                html.Div([
                    dcc.Dropdown(id='Name',
                                options = Crime_names,
                                value = 'MURDER')
                ],
                    style={"width":'48%','display':'inline-block'}
                )
                ]),
          html.Div([
              html.Div([
                      dcc.Graph(id = 'graph',figure = {})
              ],
              
                  style={"width":'48%','display':'inline-block'}
                  ),
              html.Div([
                      dcc.Graph(id='graph1',figure={})
              ],
                  style={"width":'48%','display':'inline-block'}
              )
          ])
    
])



@app.callback(
    Output('graph','figure'),
    [Input('Name_of_crime','value')]
)


def update_graph(yaxis):
    fig = px.line(df_by_year,x=df_by_year.index,y=yaxis)
    fig.update_layout(title = 'Crimes in India from 2001 to 2012',title_font_size=24,paper_bgcolor='black',
                     title_font_family='Aerial',font={'color':'red','family':'Times New Roman','size':15},
                      template = 'plotly_white')
    return fig


@app.callback(
    Output('graph1','figure'),
    [Input('Name','value')]
)


def update_graph1(yaxis):
    fig = px.bar(df_by_state,x=df_by_state.index,y=yaxis)
    fig.update_layout(title = 'Crimes in India by states',title_font_size=24,paper_bgcolor='black',
                     title_font_family='Aerial',font={'color':'red','family':'Times New Roman','size':10},
                      template = 'plotly_white')
    return fig



for running the dash board you can first copy above code in jupyter or any other IDE then click on the link http://127.0.0.1:8050/



In [ ]:
if __name__ == '__main__':
    app.run_server(port = 8050,debug=True)

# state wise 

As we can see from the dash board that the top 10 states in crimes and the state Madhya pradesh is the most dangerous state in India followed by Maharashtra Now we will see a top 10 states in India in detail

In [ ]:
df_top10 = df_by_state.sort_values('TOTAL IPC CRIMES',ascending=False).head(10)
df_top10.drop('YEAR',axis=1,inplace=True)
fig = px.pie(df_top10,values='TOTAL IPC CRIMES',names=df_top10.index,title = 'PIECHART',template='plotly_dark')
fig.update_layout(title = 'Total no of IPC crimes (%) in top 10 states',title_font_size=24,paper_bgcolor='black',
                     title_font_family='Aerial',font={'color':'red','family':'Times New Roman','size':10})
import plotly.io as pio#to display a figure defined by a dictuse the low level plotly.io.show
pio.show(fig)

Now from above pie chart we can see that Madhya pradesh is the top most in the list of crimes from 2001-2012 Now we want to analyse these top 10 states in detail. For each state we want to see in which district crimes are the most and which crime is the most popular in which district for this type of heirarchial dataset plotly express has a very good plot that is the sunburst by which we can see all the informatiion by clicking on the states in the year 2012

In [ ]:
df_state_top10 = df[(df['STATE/UT']== 'UTTAR PRADESH') | (df['STATE/UT']== 'MADHYA PRADESH') | (df['STATE/UT']== 'MAHARASHTRA') |
  (df['STATE/UT']== 'KARNATAKA') | (df['STATE/UT']== 'BIHAR') | (df['STATE/UT'] == 'GUJARAT') | (df['STATE/UT']== 'ANDHRA PRADESH') |
  (df['STATE/UT']== 'TAMIL NADU') | (df['STATE/UT']== 'KERALA')| (df['STATE/UT']== 'RAJASTHAN')]


df_top10_det = df_state_top10[df['DISTRICT'] != 'TOTAL']

df_top10_det.head()

In [ ]:
famous_crime = ['MURDER','RAPE','KIDNAPPING & ABDUCTION','ROBBERY','IMPORTATION OF GIRLS FROM FOREIGN COUNTRIES','DACOITY'
                ,'RIOTS']

fig = px.sunburst(df_top10_det[df_top10_det['YEAR'] == 2012],path = ['STATE/UT','DISTRICT'],values='TOTAL IPC CRIMES',
                  color='MURDER',hover_data = famous_crime,hover_name = 'DISTRICT',template = 'plotly_dark')
fig.update_layout(title = 'Crimes in top 10 states with their district in 2012 ',title_font_size=24,paper_bgcolor='black',
                     title_font_family='Aerial',font={'color':'red','family':'Times New Roman','size':15})

from above plot when we click on any state then we can see the in which district which crime has most popular the size of leafs are depend on the total ipc crimes and the color gives us the idea murder from above graph we can see the patna is most popular city in Murder in all the district in top 10 states in 2012 of these top10 states

In [ ]:
df_state_top10 = df_state_top10[df_state_top10['DISTRICT'] == 'TOTAL']

In [ ]:

fig = px.area(df_state_top10,x=df_state_top10['STATE/UT'],y=famous_crime,animation_frame='YEAR',template='plotly_white')

fig.update_layout(title = 'Crimes in top 10 states with their district in 2001-2012 ',title_font_size=24,paper_bgcolor='black',
                     title_font_family='Aerial',font={'color':'#e31757','family':'Times New Roman','size':11})


from above graph we can collect so much information 

# City Wise analysis

In [ ]:
df_by_city = df.groupby(by='DISTRICT').sum()
df_by_city = df_by_city[df_by_city.index != 'TOTAL']
df_by_city.drop('YEAR',axis =1,inplace=True)
df_by_city.sort_values(by='TOTAL IPC CRIMES',ascending = False,inplace=True)
total= df_by_city['TOTAL IPC CRIMES'].sum()

IPC_pct = []
for i in range(0,807):
    IPC_pct.append((df_by_city['TOTAL IPC CRIMES'][i]/total)*100)

df_by_city['IPC_pct'] = IPC_pct
df_by_city

from above dataframe we can conclude that the top most dangerous city in India is Delhi with a 2.5% of all the total ipc crimes in India the top 5 cities in crime rate are : DELHI - 2.5% , BANGLORE - 1.42% , MUMBAI - 0.906% , AHEMADABAD - 0.887143 , INDORE - 0.831771% in 2001-2012

now we will do analysis of top 10 cities in crimes of India

In [ ]:
df_by_city_top10 = df_by_city.head(10)
fig = px.sunburst(df_by_city_top10,path=[df_by_city_top10.index],values='TOTAL IPC CRIMES'
                  ,color='MURDER',hover_data=famous_crime)
fig.update_layout(title = 'Top 10 cities in crimes in India',title_font_size=24,paper_bgcolor='black',
                     title_font_family='Aerial',font={'color':'red','family':'Times New Roman','size':10},
                      template = 'plotly_white')
fig

In [ ]:
fig = px.scatter(df_by_city_top10,x=df_by_city_top10.index,y=famous_crime,template='plotly_white')

fig.update_layout(title = 'Crimes in top 10 states with their district in 2012 ',title_font_size=24,paper_bgcolor='black',
                     title_font_family='Aerial',font={'color':'#e31757','family':'Times New Roman','size':11})

fig.update_traces(marker=dict(size=12,
                              line=dict(width=2,
                                        color='DarkSlateGrey')),
                  selector=dict(mode='markers'))
fig

Since we get to know Delhi has the most criminal rates in all the cities now we will see Delhi in detail 

In [ ]:
df_Delhi = df[df['DISTRICT'] == 'DELHI UT TOTAL']
fig,ax=plt.subplots(figsize=(15,10))
plt.tick_params(labelsize=14)
plt.xticks(label = df_Delhi['YEAR'],rotation=90)
ax.bar(x=df_Delhi['YEAR'],height=df_Delhi['RIOTS'])
ax.set_facecolor('1')
ax.set_title("Bar graph of total no Riots in Delhi Vs Year",fontdict={'size':26,'fontfamily':"Britannic Bold"})
ax.set_ylabel("Total no of Riots",fontdict={'size':20,'fontfamily':"DejaVu Sans"})
ax.set_xlabel("Year",fontdict={'size':20,'fontfamily':"DejaVu Sans"})


as we can see the graph the total no of riots are maximum in 2001 because in 2001 there was an attack on parliament for more details click on the link https://en.wikipedia.org/wiki/2001_Indian_Parliament_attack 
and in India most communal incidents are in delhi

In [ ]:
fig,ax=plt.subplots(figsize=(15,10))
plt.tick_params(labelsize=14)
plt.xticks(label = df_Delhi['YEAR'],rotation=90)
ax.barh(y=df_Delhi['YEAR'],width =df_Delhi['RAPE'])
ax.set_facecolor('1')
ax.set_title("Bar graph of total no Riots in Delhi Vs Year",fontdict={'size':26,'fontfamily':"Britannic Bold"})
ax.set_ylabel("Total no of Riots",fontdict={'size':20,'fontfamily':"DejaVu Sans"})
ax.set_xlabel("Year",fontdict={'size':20,'fontfamily':"DejaVu Sans"})

As you can see the total no of rape cases in 2012 is maximum also in 2012 the Nirbhaya gang rape and murder case is also done for more details click on https://en.wikipedia.org/wiki/2012_Delhi_gang_rape_and_murder

That's all! Thanks for spending your precious time on our notebook. This is our first notebook for dataviz, we have tried our best to extract uneven points from data with the help of charts.In case we missed something important please comment it out below. Once again thank you.